In [1]:
import numpy as np
import alc

In [2]:
# Cargar datos 
train_cats = np.load('cats_and_dogs\\train\\cats\\efficientnet_b3_embeddings.npy')
train_dogs = np.load('cats_and_dogs\\train\\dogs\\efficientnet_b3_embeddings.npy')

val_cats = np.load('cats_and_dogs\\val\\cats\\efficientnet_b3_embeddings.npy')
val_dogs = np.load('cats_and_dogs\\val\\dogs\\efficientnet_b3_embeddings.npy')


In [3]:
# X_train: OK (concatenar embeddings horizontalmente)
X_train = np.zeros((train_cats.shape[0], train_cats.shape[1] + train_dogs.shape[1]))
X_train[:, :train_cats.shape[1]] = train_cats
X_train[:, train_cats.shape[1]:] = train_dogs

# X_val: OK
X_val = np.zeros((val_cats.shape[0], val_cats.shape[1] + val_dogs.shape[1]))
X_val[:, :val_cats.shape[1]] = val_cats
X_val[:, val_cats.shape[1]:] = val_dogs

# Y_train: CORREGIDO - debe ser (2, num_total_imagenes)
Y_train = np.zeros((2, train_cats.shape[1] + train_dogs.shape[1]))  # ✓ (2, 3000)
Y_train[0, :train_cats.shape[1]] = 1  # Primera fila = gatos [1,0]
Y_train[1, train_cats.shape[1]:] = 1  # Segunda fila = perros [0,1]

# Y_val: CORREGIDO - debe ser (2, num_total_imagenes)
Y_val = np.zeros((2, val_cats.shape[1] + val_dogs.shape[1]))  # ✓ (2, 2000)
Y_val[0, :val_cats.shape[1]] = 1   # Primera fila = gatos
Y_val[1, val_cats.shape[1]:] = 1   # Segunda fila = perros

print(X_train.shape, Y_train.shape)  # (1536, 3000) (2, 3000)
print(X_val.shape, Y_val.shape)      # (1536, 2000) (2, 2000)

(1536, 2000) (2, 2000)
(1536, 1000) (2, 1000)


In [4]:
W, pX = alc.fullyConectedCholesky(X_train, Y_train)

Caso (b) N < M
Calculo de Cholesky
Resolviendo sistemas triangulares .1
Resolviendo sistemas triangulares .2
Calculando W


(dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'))

In [5]:
np.save("Wchols.npy", W)
np.save("pXchols.npy", pX)



In [6]:
Wchols = W
pXchols = pX

In [9]:
Wchols.shape

(2, 1536)

In [4]:
Wchols = np.load('Wchols.npy')
pXchols = np.load('pXchols.npy')

In [6]:
alc.esPseudoInversa(X_train, pXchols, tol=1e-8)

True

In [10]:
# Evaluación con los datos de validación X_val, Y_val
W_nuevo, pX_nuevo = Wchols, pXchols

# Predección
Y_val_pred = W_nuevo @ X_val

# Cálculo de accuracy
predicciones_correctas = 0
num_muestras = Y_val.shape[1]
for i in range(num_muestras):
    clase_predicha = np.argmax(Y_val_pred[:, i])
    clase_real = np.argmax(Y_val[:, i])
    if clase_predicha == clase_real:
        predicciones_correctas += 1
accuracy = predicciones_correctas / num_muestras
print(f"\nAccuracy en datos de validación: {accuracy * 100:.2f}%")



Accuracy en datos de validación: 68.40%


In [6]:
print("=== VERIFICACIÓN COMPLETA ===")
print(f"X_train.shape: {X_train.shape}")
print(f"pXchols.shape: {pXchols.shape}")
print(f"Wchols.shape: {Wchols.shape}")

print("\n=== VERIFICANDO CONDICIONES MANUALMENTE ===")

# Verificar condición 1: X @ pX @ X = X
print("\n1. Condición 1: X @ pX @ X = X")
XpXX = X_train @ pXchols @ X_train
print(f"  X @ pX @ X shape: {XpXX.shape}")
print(f"  Primeros 5 elementos de X_train[0,:]: {X_train[0, :5]}")
print(f"  Primeros 5 elementos de (X@pX@X)[0,:]: {XpXX[0, :5]}")
diff1 = X_train - XpXX
print(f"  Norma de la diferencia: {alc.norma(diff1.flatten(), 2):.4e}")
print(f"  Norma de X_train: {alc.norma(X_train.flatten(), 2):.4e}")
print(f"  Error relativo: {alc.norma(diff1.flatten(), 2) / alc.norma(X_train.flatten(), 2):.4e}")

# Verificar condición 2: pX @ X @ pX = pX
print("\n2. Condición 2: pX @ X @ pX = pX")
pXXpX = pXchols @ X_train @ pXchols
print(f"  pX @ X @ pX shape: {pXXpX.shape}")
print(f"  Primeros 5 elementos de pX[0,:]: {pXchols[0, :5]}")
print(f"  Primeros 5 elementos de (pX@X@pX)[0,:]: {pXXpX[0, :5]}")
diff2 = pXchols - pXXpX
print(f"  Norma de la diferencia: {alc.norma(diff2.flatten(), 2):.4e}")
print(f"  Norma de pX: {alc.norma(pXchols.flatten(), 2):.4e}")
print(f"  Error relativo: {alc.norma(diff2.flatten(), 2) / alc.norma(pXchols.flatten(), 2):.4e}")

# Verificar condición 3: (X @ pX)^T = X @ pX (simetría)
print("\n3. Condición 3: (X @ pX)^T = X @ pX")
XpX = X_train @ pXchols
XpX_T = XpX.T
print(f"  X @ pX shape: {XpX.shape}")
print(f"  (X @ pX)^T shape: {XpX_T.shape}")
print(f"  Elemento [0,0]: {XpX[0, 0]:.6f} vs {XpX_T[0, 0]:.6f}")
print(f"  Elemento [0,1]: {XpX[0, 1]:.6f} vs {XpX_T[0, 1]:.6f}")
print(f"  Elemento [1,0]: {XpX[1, 0]:.6f} vs {XpX_T[1, 0]:.6f}")
diff3 = XpX.T - XpX
print(f"  Norma de la diferencia: {alc.norma(diff3.flatten(), 2):.4e}")
print(f"  Error relativo: {alc.norma(diff3.flatten(), 2) / alc.norma(XpX.flatten(), 2):.4e}")

# Verificar condición 4: (pX @ X)^T = pX @ X (simetría)
print("\n4. Condición 4: (pX @ X)^T = pX @ X")
pXX = pXchols @ X_train
pXX_T = pXX.T
print(f"  pX @ X shape: {pXX.shape}")
print(f"  (pX @ X)^T shape: {pXX_T.shape}")
print(f"  Elemento [0,0]: {pXX[0, 0]:.6f} vs {pXX_T[0, 0]:.6f}")
print(f"  Elemento [0,1]: {pXX[0, 1]:.6f} vs {pXX_T[0, 1]:.6f}")
print(f"  Elemento [1,0]: {pXX[1, 0]:.6f} vs {pXX_T[1, 0]:.6f}")
diff4 = pXX.T - pXX
print(f"  Norma de la diferencia: {alc.norma(diff4.flatten(), 2):.4e}")
print(f"  Error relativo: {alc.norma(diff4.flatten(), 2) / alc.norma(pXX.flatten(), 2):.4e}")

# Verificar que W = Y @ pX
print("\n=== VERIFICANDO W ===")
W_check = Y_train @ pXchols
print(f"  W calculado shape: {Wchols.shape}")
print(f"  Y @ pX shape: {W_check.shape}")
print(f"  ¿Son iguales? {alc.matricesIguales(Wchols, W_check)}")
if not alc.matricesIguales(Wchols, W_check):
    print(f"  ERROR: W no se calculó como Y @ pX")
    print(f"  Diferencia: {alc.norma((Wchols - W_check).flatten(), 2):.4e}")

=== VERIFICACIÓN COMPLETA ===
X_train.shape: (1536, 2000)
pXchols.shape: (2000, 1536)
Wchols.shape: (2, 1536)

=== VERIFICANDO CONDICIONES MANUALMENTE ===

1. Condición 1: X @ pX @ X = X
  X @ pX @ X shape: (1536, 2000)
  Primeros 5 elementos de X_train[0,:]: [ 0.01321056  0.39967388  0.46464351 -0.04907167 -0.03964727]
  Primeros 5 elementos de (X@pX@X)[0,:]: [ 0.01320964  0.39967386  0.46464266 -0.0490718  -0.03964957]
  Norma de la diferencia: 1.4634e+03
  Norma de X_train: 4.3432e+02
  Error relativo: 3.3693e+00

2. Condición 2: pX @ X @ pX = pX
  pX @ X @ pX shape: (2000, 1536)
  Primeros 5 elementos de pX[0,:]: [ 0.00010941 -0.00085963  0.00047834  0.00176374  0.00103985]
  Primeros 5 elementos de (pX@X@pX)[0,:]: [0.04892757 0.05676796 0.03488357 0.04684722 0.04067733]
  Norma de la diferencia: 1.3827e+01
  Norma de pX: 9.4026e+00
  Error relativo: 1.4705e+00

3. Condición 3: (X @ pX)^T = X @ pX
  X @ pX shape: (1536, 1536)
  (X @ pX)^T shape: (1536, 1536)
  Elemento [0,0]: 1.000

In [7]:
print("=== TEST CHOLESKY ===")
XXT = X_train @ X_train.T
L_chols = alc.calculaCholesky(XXT)
LLT = L_chols @ L_chols.T
error_cholesky = alc.norma((XXT - LLT).flatten(), 2) / alc.norma(XXT.flatten(), 2)
print(f"Error Cholesky: {error_cholesky:.4e}")

if error_cholesky > 1e-6:
    print("⚠️ PROBLEMA EN CHOLESKY!")

=== TEST CHOLESKY ===
Error Cholesky: 2.5525e-08


In [9]:
print("=== VERIFICACIÓN DE RANGO (con NumPy) ===")

# Verificar rango de X_train
XXT = X_train @ X_train.T
print(f"X_train.shape: {X_train.shape}")
print(f"XX^T shape: {XXT.shape}")

# Calcular autovalores con NumPy
autovalores = np.linalg.eigvalsh(XXT)  # eigvalsh para matrices simétricas
autovalores_sorted = np.sort(autovalores)[::-1]

print(f"\nPrimeros 10 autovalores (mayores):")
print(autovalores_sorted[:10])
print(f"\nÚltimos 10 autovalores (menores):")
print(autovalores_sorted[-10:])

# Contar cuántos autovalores son > 0
tol_rank = 1e-8
num_autovalores_positivos = np.sum(autovalores > tol_rank)
print(f"\nRango estimado (autovalores > {tol_rank}): {num_autovalores_positivos}")
print(f"Dimensión esperada n: {X_train.shape[0]}")

# Verificar con np.linalg.matrix_rank
rango_np = np.linalg.matrix_rank(X_train)
print(f"Rango de X_train (np.linalg.matrix_rank): {rango_np}")

if num_autovalores_positivos < X_train.shape[0]:
    print(f"\n⚠️ PROBLEMA: X_train NO tiene rango completo!")
    print(f"   Faltan {X_train.shape[0] - num_autovalores_positivos} dimensiones")
    print(f"   La matriz XX^T es singular o casi singular!")
    
# Verificar número de condición
cond_number = autovalores_sorted[0] / autovalores_sorted[-1]
print(f"\nNúmero de condición de XX^T: {cond_number:.2e}")
if cond_number > 1e10:
    print("⚠️ Matriz muy mal condicionada!")

# Comparar pseudoinversa de NumPy con la tuya
print("\n=== COMPARACIÓN CON PSEUDOINVERSA DE NUMPY ===")
pX_numpy = np.linalg.pinv(X_train)
print(f"pX NumPy shape: {pX_numpy.shape}")
print(f"pX Cholesky shape: {pXchols.shape}")

# Verificar diferencia
diff = pX_numpy - pXchols
error_diff = np.linalg.norm(diff) / np.linalg.norm(pX_numpy)
print(f"Error relativo entre pX_numpy y pXchols: {error_diff:.4e}")

if error_diff > 0.1:
    print("⚠️ GRAN DIFERENCIA entre pseudoinversas!")
    
# Verificar condiciones de Moore-Penrose con pX de NumPy
print("\n=== VERIFICAR CONDICIONES CON pX de NUMPY ===")
XpXX_np = X_train @ pX_numpy @ X_train
error1_np = np.linalg.norm(X_train - XpXX_np) / np.linalg.norm(X_train)
print(f"Condición 1 (NumPy): {error1_np:.4e}")

pXXpX_np = pX_numpy @ X_train @ pX_numpy
error2_np = np.linalg.norm(pX_numpy - pXXpX_np) / np.linalg.norm(pX_numpy)
print(f"Condición 2 (NumPy): {error2_np:.4e}")

XpX_np = X_train @ pX_numpy
error3_np = np.linalg.norm(XpX_np.T - XpX_np) / np.linalg.norm(XpX_np)
print(f"Condición 3 (NumPy): {error3_np:.4e}")

pXX_np = pX_numpy @ X_train
error4_np = np.linalg.norm(pXX_np.T - pXX_np) / np.linalg.norm(pXX_np)
print(f"Condición 4 (NumPy): {error4_np:.4e}")

=== VERIFICACIÓN DE RANGO (con NumPy) ===
X_train.shape: (1536, 2000)
XX^T shape: (1536, 1536)

Primeros 10 autovalores (mayores):
[53055.08092306  4304.29295078  3888.29878502  3511.76269982
  2916.52327841  2745.02316486  2372.01921214  2055.9235619
  1806.80531439  1556.31308355]

Últimos 10 autovalores (menores):
[0.51586721 0.50124867 0.49704126 0.4902361  0.46890068 0.44989487
 0.43529354 0.41821959 0.4033611  0.40208202]

Rango estimado (autovalores > 1e-08): 1536
Dimensión esperada n: 1536
Rango de X_train (np.linalg.matrix_rank): 1536

Número de condición de XX^T: 1.32e+05

=== COMPARACIÓN CON PSEUDOINVERSA DE NUMPY ===
pX NumPy shape: (2000, 1536)
pX Cholesky shape: (2000, 1536)
Error relativo entre pX_numpy y pXchols: 8.0661e-01
⚠️ GRAN DIFERENCIA entre pseudoinversas!

=== VERIFICAR CONDICIONES CON pX de NUMPY ===
Condición 1 (NumPy): 4.2827e-15
Condición 2 (NumPy): 7.0134e-15
Condición 3 (NumPy): 1.1331e-14
Condición 4 (NumPy): 1.3165e-14


In [10]:
print("=== DEBUG PASO A PASO ===")

# Recalcular manualmente para ver dónde falla
print("\n1. Calcular XX^T")
XXT_manual = X_train @ X_train.T
print(f"XX^T shape: {XXT_manual.shape}")

print("\n2. Cholesky de XX^T")
L = alc.calculaCholesky(XXT_manual)
print(f"L shape: {L.shape}")

print("\n3. Resolver L @ V = X")
V = alc.res_tri_matricial(L, X_train, inferior=True)
print(f"V shape: {V.shape}")

# Verificar que L @ V = X
LV = alc.prodMat(L, V)
error_LV = np.linalg.norm(X_train - LV) / np.linalg.norm(X_train)
print(f"Error L@V = X: {error_LV:.4e}")
if error_LV > 1e-6:
    print("⚠️ PROBLEMA en res_tri_matricial (paso 1)")

print("\n4. Resolver L^T @ U = V")
U = alc.res_tri_matricial(L.T, V, inferior=False)
print(f"U shape: {U.shape}")

# Verificar que L^T @ U = V
LTU = alc.prodMat(L.T, U)
error_LTU = np.linalg.norm(V - LTU) / np.linalg.norm(V)
print(f"Error L^T@U = V: {error_LTU:.4e}")
if error_LTU > 1e-6:
    print("⚠️ PROBLEMA en res_tri_matricial (paso 2)")

print("\n5. pX = U^T")
pX_manual = U.T
print(f"pX_manual shape: {pX_manual.shape}")

print("\n6. Verificar U = (XX^T)^{-1} @ X")
# U debería ser (XX^T)^{-1} @ X
XXT_inv_X_np = np.linalg.solve(XXT_manual, X_train)
error_U = np.linalg.norm(U - XXT_inv_X_np) / np.linalg.norm(XXT_inv_X_np)
print(f"Error U vs (XX^T)^{-1}@X: {error_U:.4e}")
if error_U > 1e-6:
    print("⚠️ PROBLEMA: U no es (XX^T)^{-1} @ X")

print("\n7. Comparar con NumPy")
error_vs_numpy = np.linalg.norm(pX_manual - pX_numpy) / np.linalg.norm(pX_numpy)
print(f"Error pX_manual vs pX_numpy: {error_vs_numpy:.4e}")

print("\n8. Verificar que tu función devuelve lo mismo")
print(f"Error pXchols vs pX_manual: {np.linalg.norm(pXchols - pX_manual):.4e}")

=== DEBUG PASO A PASO ===

1. Calcular XX^T
XX^T shape: (1536, 1536)

2. Cholesky de XX^T
L shape: (1536, 1536)

3. Resolver L @ V = X
V shape: (1536, 2000)
Error L@V = X: 9.5367e-16

4. Resolver L^T @ U = V
U shape: (1536, 2000)
Error L^T@U = V: 4.0185e-15

5. pX = U^T
pX_manual shape: (2000, 1536)

6. Verificar U = (XX^T)^{-1} @ X
Error U vs (XX^T)^-1@X: 1.2631e-05
⚠️ PROBLEMA: U no es (XX^T)^{-1} @ X

7. Comparar con NumPy
Error pX_manual vs pX_numpy: 1.2631e-05

8. Verificar que tu función devuelve lo mismo
Error pXchols vs pX_manual: 1.2831e+01


In [11]:
print("=== COMPARACIÓN: función vs manual ===")

# Ver qué tiene pXchols
print(f"\npXchols primeros 5 elementos [0,:]: {pXchols[0, :5]}")
print(f"pX_manual primeros 5 elementos [0,:]: {pX_manual[0, :5]}")
print(f"pX_numpy primeros 5 elementos [0,:]: {pX_numpy[0, :5]}")

# Mostrar tu código actual de fullyConectedCholesky
print("\n=== CÓDIGO ACTUAL ===")
print("Muéstrame las líneas exactas del caso (b) en fullyConectedCholesky")

=== COMPARACIÓN: función vs manual ===

pXchols primeros 5 elementos [0,:]: [ 0.00010941 -0.00085963  0.00047834  0.00176374  0.00103985]
pX_manual primeros 5 elementos [0,:]: [ 0.00129741 -0.00759465  0.00179526  0.0010241   0.01544648]
pX_numpy primeros 5 elementos [0,:]: [ 0.00129735 -0.00759467  0.00179516  0.00102408  0.01544641]

=== CÓDIGO ACTUAL ===
Muéstrame las líneas exactas del caso (b) en fullyConectedCholesky


In [12]:
W_nuevo, pX_nuevo = Wchols, pXchols

print(f"\nComparación:")
print(f"pX_nuevo [0,:5]: {pX_nuevo[0, :5]}")
print(f"pX_manual [0,:5]: {pX_manual[0, :5]}")  
print(f"pX_numpy [0,:5]: {pX_numpy[0, :5]}")

# Verificar que ahora sí son iguales
error_nuevo = np.linalg.norm(pX_nuevo - pX_numpy) / np.linalg.norm(pX_numpy)
print(f"\nError pX_nuevo vs pX_numpy: {error_nuevo:.4e}")

if error_nuevo < 1e-4:
    print("✓ ¡FUNCIÓN CORREGIDA EXITOSAMENTE!")
    
    # Verificar Moore-Penrose
    print("\n=== VERIFICACIÓN MOORE-PENROSE ===")
    resultado = alc.esPseudoInversa(X_train, pX_nuevo, tol=1e-4)
    print(f"¿Es pseudoinversa? {resultado}")
else:
    print("⚠️ Todavía hay diferencias, hay otro bug")


Comparación:
pX_nuevo [0,:5]: [ 0.00010941 -0.00085963  0.00047834  0.00176374  0.00103985]
pX_manual [0,:5]: [ 0.00129741 -0.00759465  0.00179526  0.0010241   0.01544648]
pX_numpy [0,:5]: [ 0.00129735 -0.00759467  0.00179516  0.00102408  0.01544641]

Error pX_nuevo vs pX_numpy: 8.0661e-01
⚠️ Todavía hay diferencias, hay otro bug
